Alexnet model

In [ ]:

# This is the code for the Alexnet where the weights of the model are not trained

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper_parameters
num_epochs = 2
learning_rate = 0.001
bs = 100

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.layer1 = nn.Conv2d(3, 96, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer2 = nn.Conv2d(96, 256, kernel_size=3, padding= 'same')
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer3 = nn.Conv2d(256, 384, kernel_size=3, padding= 'same')
        self.relu3 = nn.ReLU()

        self.layer4 = nn.Conv2d(384, 384, kernel_size=3, padding= 'same')
        self.relu4 = nn.ReLU()

        self.layer5 = nn.Conv2d(384, 256, kernel_size=3, padding= 'same')
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.flatten = nn.Flatten()
        # Calculate the input size for the fully connected layer
        self.fc1_input_size = 6*6*256
        self.fc1 = nn.Linear(self.fc1_input_size, 4096)
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(4096, 4096)
        self.relu4 = nn.ReLU()

        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.layer1(x)))
        x = self.pool2(self.relu2(self.layer2(x)))
        x = self.relu3(self.layer3(x))
        x = self.relu4(self.layer4(x))
        x = self.pool5(self.relu5(self.layer5(x)))
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

# CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = bs, shuffle=False)

# Create the model, loss function, and optimizer
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Training the model
for epoch in range(num_epochs):

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'Loss': loss.item()}, refresh=True)

    progress_bar.close()

# Evaluation
model.eval()
correct_predictions = 0
total_predictions = 0
class_correct = torch.zeros(10)
class_total = torch.zeros(10)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        c = (predicted == labels).squeeze()
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Print class-wise accuracy
for i in range(10):
    accuracy = (class_correct[i] / class_total[i]) * 100 if class_total[i] != 0 else 0
    print(f"Class {i} Accuracy: {accuracy}%")

# Print overall accuracy
overall_accuracy = (correct_predictions / total_predictions) * 100
print(f"Overall Test Accuracy: {overall_accuracy}%")

Resnet model

In [ ]:

# This is the code for a Resnet model which is not trained

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper_parameters
num_epochs = 2
learning_rate = 0.001
bs = 100

# Define the basic residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection for identity mapping when the input and output dimensions are the same
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += residual  # Add the shortcut connection
        x = self.relu(x)
        return x

# Define the ResNet architecture
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = bs, shuffle=False)

# Create the model, loss function, and optimizer
model = ResNet(ResidualBlock, [2, 2, 2])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Training the model
for epoch in range(num_epochs):

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'Loss': loss.item()}, refresh=True)

    progress_bar.close()

# Evaluation
model.eval()
correct_predictions = 0
total_predictions = 0
class_correct = torch.zeros(10)
class_total = torch.zeros(10)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        c = (predicted == labels).squeeze()
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Class-wise accuracy
for i in range(10):
    accuracy = (class_correct[i] / class_total[i]) * 100 if class_total[i] != 0 else 0
    print(f"Class {i} Accuracy: {accuracy}%")

# Overall accuracy
overall_accuracy = (correct_predictions / total_predictions) * 100
print(f"Overall Test Accuracy: {overall_accuracy}%")


Now, to compare the results of the above 2 models, employ the below code, which will demonstrate the training and test accuracies of the model.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

#Device Configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Hyperparameters
batch_size = 32
learning_rate = 0.001
num_epochs = 5

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Lists to store accuracy values during training and evaluation
alexnet_train_accuracy = []
resnet_train_accuracy = []
alexnet_test_accuracy = []
resnet_test_accuracy = []

# Function to train the model
def train_model(model, criterion, optimizer, scheduler, train_accuracy_list, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        correct_predictions = 0
        total_predictions = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Track training accuracy
            _, preds = torch.max(outputs, 1)
            correct_predictions += (preds == labels).sum().item()
            total_predictions += labels.size(0)

        train_accuracy = correct_predictions / total_predictions
        train_accuracy_list.append(train_accuracy)
        progress_bar.close()

        # Evaluation on the validation set
        model.eval()
        correct_predictions = 0
        total_predictions = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                correct_predictions += (preds == labels).sum().item()
                total_predictions += labels.size(0)

        test_accuracy = correct_predictions / total_predictions
        scheduler.step()

        if isinstance(model, AlexNet):
            alexnet_test_accuracy.append(test_accuracy)
        elif isinstance(model, ResNet):
            resnet_test_accuracy.append(test_accuracy)

# Model implementation (AlexNet)
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Conv2d(3, 96, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer2 = nn.Conv2d(96, 256, kernel_size=3, padding= 'same')
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer3 = nn.Conv2d(256, 384, kernel_size=3, padding= 'same')
        self.relu3 = nn.ReLU()

        self.layer4 = nn.Conv2d(384, 384, kernel_size=3, padding= 'same')
        self.relu4 = nn.ReLU()

        self.layer5 = nn.Conv2d(384, 256, kernel_size=3, padding= 'same')
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.flatten = nn.Flatten()

        # Calculating the input size for the fully connected layer
        self.fc1_input_size = 6*6*256
        self.fc1 = nn.Linear(self.fc1_input_size, 4096)
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(4096, 4096)
        self.relu4 = nn.ReLU()

        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = self.pool1(self.relu1(self.layer1(x)))
        x = self.pool2(self.relu2(self.layer2(x)))
        x = self.relu3(self.layer3(x))
        x = self.relu4(self.layer4(x))
        x = self.pool5(self.relu5(self.layer5(x)))
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        return x

# Model implementation (ResNet)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut connection for identity mapping when the input and output dimensions are the same
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += residual  # Add the shortcut connection
        x = self.relu(x)
        return x

# Define the ResNet architecture
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(block(out_channels, out_channels, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Instantiate the models
alexnet_model = AlexNet().to(device)
model = ResNet(ResidualBlock, [2, 2, 2])
resnet_model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
alexnet_optimizer = optim.Adam(alexnet_model.parameters(), lr=learning_rate)
resnet_optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)

# Learning rate scheduler
alexnet_scheduler = lr_scheduler.StepLR(alexnet_optimizer, step_size=5, gamma=0.1)
resnet_scheduler = lr_scheduler.StepLR(resnet_optimizer, step_size=5, gamma=0.1)

# Train the models
train_model(alexnet_model, criterion, alexnet_optimizer, alexnet_scheduler, alexnet_train_accuracy, num_epochs)
train_model(resnet_model, criterion, resnet_optimizer, resnet_scheduler, resnet_train_accuracy, num_epochs)

# Plot the training and test accuracy curves
plt.figure(figsize=(12, 6))

# AlexNet
plt.subplot(1, 2, 1)
plt.plot(range(1, num_epochs + 1), alexnet_train_accuracy, label='Train Accuracy')
plt.plot(range(1, num_epochs + 1), alexnet_test_accuracy, label='Test Accuracy')
plt.title('AlexNet Training and Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# ResNet
plt.subplot(1, 2, 2)
plt.plot(range(1, num_epochs + 1), resnet_train_accuracy, label='Train Accuracy')
plt.plot(range(1, num_epochs + 1), resnet_test_accuracy, label='Test Accuracy')
plt.title('ResNet Training and Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


Below one is the code for simulating a pre trained Alexnet architecture

In [ ]:
# This is the code for simulating a pre trained Alexnet architecture

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

from torchinfo import summary
from tqdm import tqdm

#Hyper_parameters
num_epochs = 2
bs = 32  #Batch size
learning_rate = 0.001

#Device Configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Data transformations for CIFAR10
transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
weights = models.AlexNet_Weights.DEFAULT
weights.transforms()

#CIFAR10 dataset and dataloaders
training_dataset = datasets.CIFAR10('./data_src', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10('./data_src', train=False, download=True, transform=transform)

train_loader = DataLoader(training_dataset, batch_size=bs, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False, drop_last=True)

#Loading the pretrained AlexNet Model
AlexneT = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)
model = AlexneT.to(device)
summary(model, input_size=(32, 3, 224, 224))

# Here, as the dataset taken has 10 classes, so the model should also be calebrited accourdingly. For this, last set of fully connected layers are modified to give the output in 10.
modified_AlexNet = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# As we only need to train the last layer for the modifications, thus, freezing rest of the layers.
for param in modified_AlexNet.parameters():
    param.requires_grad = False

# Modifications applied
modified_AlexNet.avgpool = nn.AdaptiveAvgPool2d(output_size=(2, 2))
modified_AlexNet.classifier = nn.Sequential()
modified_AlexNet.classifier.add_module('dropout1', nn.Dropout(p=0.5))
modified_AlexNet.classifier.add_module('linear1', nn.Linear(in_features=1024, out_features=512))
modified_AlexNet.classifier.add_module('relu1', nn.ReLU(inplace=True))
modified_AlexNet.classifier.add_module('dropout2', nn.Dropout(p=0.5))
modified_AlexNet.classifier.add_module('linear2', nn.Linear(in_features=512, out_features=512))
modified_AlexNet.classifier.add_module('relu2', nn.ReLU(inplace=True))
modified_AlexNet.classifier.add_module('dropout3', nn.Dropout(p=0.5))
modified_AlexNet.classifier.add_module('linear3', nn.Linear(in_features=512, out_features=10)) #Output features changed from 1000 to 10.

# Summary of the modified model
model_to_train = modified_AlexNet.to(device)
summary(model_to_train, input_size=(32, 3, 224, 224))

# Loss and optimizer functions
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_to_train.parameters(), lr= learning_rate)

# Training the model
for epoch in range(num_epochs):

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_to_train(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'Loss': loss.item()}, refresh=True)

    progress_bar.close()

# Evaluation
model_to_train.eval()
correct_predictions = 0
total_predictions = 0
class_correct = torch.zeros(10)
class_total = torch.zeros(10)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model_to_train(images)
        _, predicted = torch.max(outputs, 1)

        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        c = (predicted == labels).squeeze()
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Class wise accuracy
for i in range(10):
    accuracy = (class_correct[i] / class_total[i]) * 100 if class_total[i] != 0 else 0
    print(f"Class {i} Accuracy: {accuracy}%")

# Overall accuray of the model
overall_accuracy = (correct_predictions / total_predictions) * 100
print(f"Overall Test Accuracy: {overall_accuracy}%")


Below one is the code for the simulation of a pre-trained Resnet architiecture


In [ ]:

# This is the code for the simulation of a pretrained Resnet architiecture

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

from torchinfo import summary
from tqdm import tqdm


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Hyper_parameters
num_epochs = 2
learning_rate = 0.001
bs = 32

# CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = bs, shuffle=False)

# Loading the pretrained ResNet18 architecture
ResNet = models.resnet18(pretrained = True)
model = ResNet.to(device)
summary(model, input_size=(32, 3, 32, 32))

# Here, as the dataset taken has 10 classes, so the model should also be calebrited accourdingly. For this, last set of fully connected layers are modified to give the output in 10.
modified_model = models.resnet18(pretrained = True)

# As we only need to train the last layer for the modifications, thus, freezing rest of the layers.
for param in modified_model.parameters():
    param.requires_grad = False

# Modifications
modified_fc = nn.Linear(in_features=512, out_features=10, bias=True)
modified_model.fc = modified_fc

# Summary of the modified model
model_to_train = modified_model.to(device)
summary(model_to_train, input_size=(32, 3, 32, 32))

# Loss and optimizer functions
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_to_train.parameters(), lr=0.001)

# Training the model
for epoch in range(num_epochs):

    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False)
    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_to_train(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'Loss': loss.item()}, refresh=True)

    progress_bar.close()

# Evaluation
model_to_train.eval()
correct_predictions = 0
total_predictions = 0
class_correct = torch.zeros(10)
class_total = torch.zeros(10)

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model_to_train(images)
        _, predicted = torch.max(outputs, 1)

        total_predictions += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()

        c = (predicted == labels).squeeze()
        for i in range(10):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

# Class-wise accuracy
for i in range(10):
    accuracy = (class_correct[i] / class_total[i]) * 100 if class_total[i] != 0 else 0
    print(f"Class {i} Accuracy: {accuracy}%")

# Overall accuracy of the model
overall_accuracy = (correct_predictions / total_predictions) * 100
print(f"Overall Test Accuracy: {overall_accuracy}%")
